In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sklearn as sk
from sklearn import decomposition as dec
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.utils.prune as prune
import  sklearn.manifold as nonlin


In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

mnist_train = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

trainloader_mnist =  torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader_mnist =  torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                          shuffle=True, num_workers=2)



                                        


Files already downloaded and verified
Files already downloaded and verified


# DImensional Reduction on mnist dataset

In [73]:
class NN(nn.Module):
    def __init__(self, is_normalize):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=12, kernel_size=(5, 5))
        self.pool = nn.MaxPool2d(kernel_size=(2), stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=12, out_channels=16, kernel_size=(5, 5))
        self.fc1 = nn.Linear(256, 120) # 256x 60
        self.fc2 = nn.Linear(120, 84) # 60 x 84
        self.fc3 = nn.Linear(84, 10)
        self.soft = nn.Softmax(dim=0)
        self.norm = is_normalize

        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        if(self.norm==True):
         x = nn.functional.normalize(self.fc3(x))
        else:
         x = self.fc3(x)
         
        # x = self.soft(x)
        return x
    

In [92]:
# path_mnist = './isomap_reduced_network/custom_nn_mnist_isomap_original.pt'
# model_mnist = torch.load(path_mnist)
model_mnist = NN(is_normalize=False)
model_mnist.eval()


NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [4]:
def are_weights(model,weights):
    w1,w2,w3 = weights
    model.fc1.weight.data = w1
    model.fc2.weight.data = w2
    model.fc3.weight.data = w3
    return model
     

In [5]:
def construct_model(model,layers):
    l1,l2,l3 = layers
    model.fc1 = l1
    model.fc2 = l2
    model.fc3 = l3
    return model

In [98]:
layers = (nn.Linear(256, 60), nn.Linear(60, 42),nn.Linear(42, 10))
small_network_scratch = construct_model(NN(is_normalize=False), layers)
small_network_scratch.eval()

NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=42, bias=True)
  (fc3): Linear(in_features=42, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [4]:
random = torch.randn(256)

In [6]:
mnist_dataset_sample = iter(testloader_mnist)
sample_im, sample_lbl = mnist_dataset_sample.next()


In [7]:
cifar_dataset_sample = iter(testloader)
cifar_sample_im, cifar_sample_lbl = cifar_dataset_sample.next()

## Matrix multiplication with non linear dimensioal reduction


In [12]:
print(model_mnist.fc1.weight.shape)

torch.Size([120, 256])


In [95]:
x_new_1 = model_mnist.fc1.weight.detach().numpy().T
iso = nonlin.Isomap(n_components=60)
# x_new_1 = x_new_1.reshape(60,2)
x_new_1 = iso.fit_transform(x_new_1)
# x_new_1 = x_new_1.flatten()
# random_matrix = random.reshape(256, 1)
# x_new_1_matrix = x_new_1.reshape(1,60)
print("x_new vector")
print(x_new_1.shape)

# new_weight_matrixIguess = np.matmul(random_matrix, x_new_1_matrix)






uuh = torch.mm(model_mnist.fc1.weight, torch.tensor(x_new_1, dtype=torch.float))
print(uuh.shape)
wfc2 = torch.mm(model_mnist.fc2.weight, uuh)
print(wfc2.shape)


blah = nonlin.Isomap(n_components=21)
wfc2_r = wfc2.reshape(42, -1)
wfc2_r = blah.fit_transform(wfc2_r.detach().numpy().T).T
wfc2_r = wfc2_r.reshape(-1, 42)
wfc2_r = torch.tensor(wfc2_r, dtype=torch.float)
wfc2_r = wfc2_r.T
print(wfc2_r.shape)


wfc3_r = torch.mm(wfc2_r, wfc2.T)
print(wfc3_r.shape)


wfc3_r = torch.mm(model_mnist.fc3.weight, wfc3_r.T)
print(wfc3_r.shape)


x_new vector
(256, 60)
torch.Size([120, 60])
torch.Size([84, 60])
torch.Size([42, 60])
torch.Size([42, 84])
torch.Size([10, 42])


In [96]:
layers2 = (nn.Linear(256, 60), nn.Linear(60, 42), nn.Linear(42,10))
dr_weights = (torch.tensor(x_new_1.T, dtype=torch.float32), wfc2_r, wfc3_r)
dr_n_projection = construct_model(NN(is_normalize=True), layers2)
dr_n_projection = are_weights(dr_n_projection, dr_weights)
print(dr_n_projection.fc1.weight.shape)
dr_n_projection.eval()

torch.Size([60, 256])


NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=42, bias=True)
  (fc3): Linear(in_features=42, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [97]:
layers3 = (nn.Linear(256, 60), nn.Linear(60, 42), nn.Linear(42,10))
dr_weights2 = (torch.tensor(x_new_1.T, dtype=torch.float32), wfc2_r, wfc3_r)
dr_n_projection_untrained = construct_model(NN(is_normalize=True), layers3)
dr_n_projection_untrained = are_weights(dr_n_projection_untrained, dr_weights2)
dr_n_projection_untrained.eval()

NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=42, bias=True)
  (fc3): Linear(in_features=42, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

## Matrix multiplication and Isomap

In [76]:
reduced_mnist = NN()
original_mnist = NN()

reduced_mnist.fc1 = nn.Linear(256, 60, bias=False)
reduced_mnist.fc2 = nn.Linear(60, 42, bias=False)
reduced_mnist.fc3 = nn.Linear(42, 10, bias=False)
reduced_mnist.fc1.weight.data = torch.tensor(x_new_1.T, dtype=torch.float32)
reduced_mnist.fc2.weight.data = wfc2_r
reduced_mnist.fc3.weight.data = wfc3_r

original_mnist.fc1.weight = model_mnist.fc1.weight
original_mnist.fc2.weight = model_mnist.fc2.weight
original_mnist.fc3.weight = model_mnist.fc3.weight

output_r = reduced_mnist(sample_im)
output = original_mnist(sample_im)


o_reduced = nn.functional.normalize(output_r, dim=0)
o_oroginal = nn.functional.normalize(output, dim=0)
print('reduced')
print(o_reduced)
print('original')
print(o_oroginal)

dist = torch.pairwise_distance(o_reduced, o_oroginal)
print(dist)
ans = 1/(dist + 1) * 100
print('pairwise distance similarity')
print(ans)

print("cosine similarity")
cos = nn.CosineSimilarity(dim=0)
print(cos(o_reduced, o_oroginal) * 100)


reduced
tensor([[-0.2288,  0.4276, -0.4188,  0.3702, -0.4817,  0.6217,  0.4096,  0.2572,
         -0.5890, -0.5364],
        [-0.7083,  0.5576,  0.5830,  0.6592, -0.5406,  0.0350, -0.6322,  0.7268,
         -0.3753, -0.4963],
        [-0.2731,  0.5499, -0.5737,  0.4457, -0.5606,  0.7816,  0.5207,  0.2788,
         -0.6892, -0.6406],
        [-0.6094, -0.4515,  0.3945,  0.4793, -0.4018,  0.0360, -0.4018,  0.5726,
         -0.1929, -0.2358]], grad_fn=<DivBackward0>)
original
tensor([[-0.3579,  0.3697, -0.4073, -0.3067, -0.3129,  0.5719,  0.0533, -0.1484,
          0.0669, -0.2238],
        [-0.6358,  0.2004, -0.5321, -0.2661, -0.0018,  0.7855, -0.1828,  0.1983,
         -0.0725, -0.3087],
        [-0.1548,  0.0426, -0.2384, -0.3687, -0.2034,  0.5129,  0.1457, -0.1748,
          0.1315, -0.4566],
        [-0.7249,  0.5352, -0.3022, -0.3997,  0.0129,  0.4900, -0.3702,  0.5585,
         -0.3119, -0.3262]], grad_fn=<AddmmBackward0>)
tensor([1.1527, 1.9220, 1.5168, 1.6268], grad_fn=<NormBackw

In [51]:
x_new_1 = model_mnist.fc1(random)
x_new = model_mnist.fc2(x_new_1)
x_new = model_mnist.fc3(x_new)
print(x_new)


tensor([-0.5093, 10.7150, -0.0736, -8.4573,  7.4311, -0.9365, -1.6053,  2.1383,
        -8.8714, -2.8670], grad_fn=<AddBackward0>)


In [57]:
fc_1 = model_mnist.fc1.weight.detach().numpy().T
fa = nonlin.Isomap(n_components=60, n_neighbors=10)
fc_1_reduced = fa.fit_transform(fc_1)
fc_1_reduced = torch.tensor(fc_1_reduced.T,  dtype=torch.float32)
print(fc_1_reduced.shape)




torch.Size([60, 256])


In [54]:
# fc_2 = nn.functional.relu(model_mnist.fc2.weight)
fc_2 = model_mnist.fc2.weight.detach().numpy()
fc_2 = fc_2.reshape(-1,60)
isomap_ = nonlin.Isomap(n_components=30, n_neighbors=5)
fc_2_reduced = isomap_.fit_transform(fc_2)
fc_2_reduced = fc_2_reduced.reshape(-1, 120)
fa_ouput = nonlin.Isomap(n_components=21)
fc_2_reduced = fa_ouput.fit_transform(fc_2_reduced.T)
fc_2_reduced = fc_2_reduced.reshape(-1, 42)
fc_1_reduced = fc_2_reduced.T
print('second dimension reduced:')

fc_2_reduced = torch.tensor(fc_2_reduced, dtype=torch.float32)
print(fc_2_reduced.shape)



second dimension reduced:
torch.Size([60, 42])


## Testing just dimensional reduction 

In [55]:
fc_3 =  model_mnist.fc3.weight.detach().numpy().T
fc3_Isomap = nonlin.Isomap(n_components=5, n_neighbors=10)
fc_3_reduced = fc3_Isomap.fit_transform(fc_3)
fc_3_reduced = fc_3_reduced.reshape(-1, 10)
fc_3_reduced = torch.tensor(fc_3_reduced.T, dtype=torch.float32)
print(fc_3_reduced.shape)


torch.Size([10, 42])


In [59]:
model_mnist.fc1 = nn.Linear(256, 60,bias=True)
model_mnist.fc2 = nn.Linear(60, 42,bias=True)
model_mnist.fc3 = nn.Linear(42, 10,bias=True)

model_mnist.fc1.weight = nn.Parameter(fc_1_reduced)
model_mnist.fc2.weight = nn.Parameter(fc_2_reduced.T)
model_mnist.fc3.weight = nn.Parameter(fc_3_reduced)


In [27]:
x_new_2 = model_mnist.fc1(random)
x_new_2 = model_mnist.fc2(x_new_2)
x_new_2 = model_mnist.fc3(x_new_2)
print(x_new_2)


tensor([  8.2577, -18.1993,   9.7784,   1.0847, -15.9523,   6.3733,  26.0245,
        -10.8813,  -0.4002, -10.5237], grad_fn=<AddBackward0>)


In [59]:
model_mnist.eval()

NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [8]:
from torch.profiler import profile, record_function, ProfilerActivity
import torch.optim as optim




def training(model, trainset,n):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    running_loss = 0.0
    total = 0.0
    correct = 0.0
    loss_nodr =[]
    acc_nodr = []

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    print(optimizer)    


    for epoch in range(n):
        
        print('epoch:', epoch)
        for i, data in enumerate(trainset):
            
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad(True)
            with torch.set_grad_enabled(True):
                with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
                 outputs = model(inputs).to(device)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            running_loss += loss.item()
            if i % 2000 == 1999:
                loss_temp = running_loss / 2000
                acc_temp = 100 * correct / total
                loss_nodr.append(loss_temp)
                acc_nodr.append(acc_temp)
                print(
                    f'[{epoch + 1}, {i + 1:5d}] loss: {loss_temp:.3f} acc: {acc_temp:.3f}')
                running_loss = 0.0
                correct = 0.0
                total = 0.0
    print('Finished Training')


## Model training and saving

### Training original model mnist dataset

In [93]:
training(model_mnist, trainloader_mnist, 2)
path_mnist_original_network = './isomap_reduced_network/custom_nn_mnist_isomap_original.pt'
torch.save(model_mnist, path_mnist_original_network)




SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0
[1,  2000] loss: 1.156 acc: 62.487
[1,  4000] loss: 0.202 acc: 93.675
[1,  6000] loss: 0.146 acc: 95.175
[1,  8000] loss: 0.118 acc: 96.425
[1, 10000] loss: 0.101 acc: 96.875
[1, 12000] loss: 0.092 acc: 97.300
[1, 14000] loss: 0.084 acc: 97.513
epoch: 1
[2,  2000] loss: 0.099 acc: 97.925
[2,  4000] loss: 0.064 acc: 98.050
[2,  6000] loss: 0.062 acc: 98.062
[2,  8000] loss: 0.062 acc: 98.162
[2, 10000] loss: 0.063 acc: 98.075
[2, 12000] loss: 0.053 acc: 98.388
[2, 14000] loss: 0.047 acc: 98.638
Finished Training


### Retraining dimensionally reduced network

In [100]:
training(dr_n_projection, trainloader_mnist, 2)
path_isomap_retrained = './isomap_reduced_network/reduced_isomap_retrained.pt'
torch.save(dr_n_projection, path_isomap_retrained)


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 1.988 acc: 42.712
[1,  4000] loss: 1.893 acc: 59.513
[1,  6000] loss: 1.876 acc: 65.325
[1,  8000] loss: 1.848 acc: 75.250
[1, 10000] loss: 1.838 acc: 76.750
[1, 12000] loss: 1.833 acc: 78.425
[1, 14000] loss: 1.825 acc: 80.125
epoch: 1
[2,  2000] loss: 2.729 acc: 81.225
[2,  4000] loss: 1.814 acc: 81.688
[2,  6000] loss: 1.806 acc: 82.800
[2,  8000] loss: 1.808 acc: 82.675
[2, 10000] loss: 1.803 acc: 83.675
[2, 12000] loss: 1.803 acc: 83.975
[2, 14000] loss: 1.798 acc: 83.612
Finished Training


In [101]:
#do the training
training(small_network_scratch,trainloader_mnist, 5)
path_small_network = './isomap_reduced_network/small_network_scratch_isomap.pt'
torch.save(small_network_scratch, path_small_network)


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0
[1,  2000] loss: 1.160 acc: 62.250
[1,  4000] loss: 0.183 acc: 94.588
[1,  6000] loss: 0.145 acc: 95.725
[1,  8000] loss: 0.117 acc: 96.487
[1, 10000] loss: 0.098 acc: 96.963
[1, 12000] loss: 0.085 acc: 97.250
[1, 14000] loss: 0.078 acc: 97.700
epoch: 1
[2,  2000] loss: 0.094 acc: 98.025
[2,  4000] loss: 0.057 acc: 98.175
[2,  6000] loss: 0.064 acc: 98.100
[2,  8000] loss: 0.062 acc: 98.200
[2, 10000] loss: 0.058 acc: 98.200
[2, 12000] loss: 0.053 acc: 98.400
[2, 14000] loss: 0.058 acc: 98.200
epoch: 2
[3,  2000] loss: 0.069 acc: 98.658


KeyboardInterrupt: 

In [83]:
averge_accuracy(model_mnist, testloader_mnist)

average acc: 98.64


In [103]:
path_small_isomap_untrained = './isomap_reduced_network/untrained_reduced_network.pt'
torch.save(dr_n_projection_untrained, path_small_isomap_untrained)

In [9]:
def averge_accuracy(model,test_dataset):
    total = 0
    correct = 0
    for data in test_dataset:
        pics, lables = data
        output = model(pics)
        _, pred = torch.max(output.data, 1)
        correct += (pred == lables).sum().item()
        total += lables.size(0)

    print('average acc:', 100 * correct / total)


In [33]:
averge_accuracy(dr_n_projection_untrained, testloader_mnist)

average acc: 11.89


In [79]:
def mfun(matrix, weight_matrix):
    for a in range(matrix.shape[0]):
        for b in range(matrix.shape[1]):
            matrix[a][b] = weight_matrix[a][b]
    return matrix


In [81]:
m1 = torch.zeros([60, 256])
m2 = torch.zeros([42, 60])
m3 = torch.zeros([10, 42])

m1 = mfun(m1, model_mnist.fc1.weight)
m2 = mfun(m2, model_mnist.fc2.weight)
m3 = mfun(m3, model_mnist.fc3.weight)

print(m1.shape)



torch.Size([60, 256])


In [87]:
lin1 = nn.Linear(in_features = 256, out_features=60,bias=True)
lin2 = nn.Linear(in_features = 60, out_features=42,bias=True)
lin3 = nn.Linear(in_features = 42, out_features=10,bias=True)

lin1.weight = model_mnist.fc1.weight
lin2.weight = model_mnist.fc2.weight
lin3.weight = model_mnist.fc3.weight

x_lin = lin1(random)
x_lin = lin2(x_lin)
x_lin = lin3(x_lin)
norm_xlin = nn.functional.normalize(x_lin, dim=0)
print(norm_xlin)
print(x_new)
ed = torch.pairwise_distance(norm_xlin, x_new)
print("distance")
print(ed)
result = 1/(1 + ed)
print(result * 100)
print("cosine similarity")
print(nn.CosineSimilarity(dim=0)(norm_xlin, x_new))



tensor([-0.1890, -0.5867,  0.5877, -0.1414, -0.0527, -0.2899,  0.3277,  0.0784,
         0.2266,  0.0545], grad_fn=<DivBackward0>)
tensor([-0.5093, 10.7150, -0.0736, -8.4573,  7.4311, -0.9365, -1.6053,  2.1383,
        -8.8714, -2.8670], grad_fn=<AddBackward0>)
distance
tensor(18.7919, grad_fn=<NormBackward1>)
tensor(5.0526, grad_fn=<MulBackward0>)
cosine similarity
tensor(-0.4186, grad_fn=<SumBackward1>)


In [81]:
lin1 = nn.Linear(in_features = 256, out_features=120,bias=False)
lin2 = nn.Linear(in_features = 120, out_features=84,bias=False)
lin3 = nn.Linear(in_features = 84, out_features=10,bias=False)

lin1.weight = model_mnist.fc1.weight
lin2.weight =  model_mnist.fc2.weight
lin3.weight =  model_mnist.fc3.weight

In [82]:
x_lin = lin1(random)
x_lin = lin2(x_lin)
x_lin = lin3(x_lin)
print(x_lin)


tensor([-0.9922,  1.0499,  1.0739, -2.9549,  1.8736,  0.2282,  0.8645,  0.9884,
         0.8530, -2.6438], grad_fn=<MvBackward0>)


## weight matrix multiplication testing

In [38]:
fc_1 = model_mnist.fc1.weight.detach().numpy().T
fa =nonlin.Isomap(n_components=60)
fc_1_m = fa.fit_transform(fc_1)
fc_1_m = torch.tensor(fc_1_m.T,  dtype=torch.float32)
print(fc_1_m.shape)



torch.Size([40, 256])


In [29]:
fc_1_m_new = torch.mm(model_mnist.fc1.weight.T, fc_1_m)
print(fc_1_m_new.shape)

torch.Size([256, 60])


In [39]:
# fc_2 = nn.functional.relu(model_mnist.fc2.weight)
# fc_2_x = torch.mm(new_matrix, model_mnist.fc2.weight.T)
# print(fc_2_x.shape)
# fc_2 = fc_2_x.detach().numpy()
# isomap_ = dec.PCA(n_components=42)
# fc_2_reduced_m = isomap_.fit_transform(fc_2)
# fc_2_reduced_m = fc_2_reduced_m.T
# print('second dimension reduced:')

# fc_2_reduced_m = torch.tensor(fc_2_reduced_m, dtype=torch.float32)
# print(fc_2_reduced_m.shape)
fc_2 = model_mnist.fc2.weight.detach().numpy()
fc_2 = fc_2.reshape(84 * 2, -1)
isomap_ = nonlin.Isomap(n_components=20)
reduced_fc_2 = isomap_.fit_transform(fc_2)
reduced_fc_2 = reduced_fc_2.reshape(-1, 20 * 2)
reduced_fc_2 = torch.tensor(reduced_fc_2,  dtype=torch.float32)
print(reduced_fc_2.shape)



torch.Size([84, 40])


In [45]:
lin_60 = nn.Linear(256, 40, bias=False)
lin_8460 = nn.Linear(40,84, bias=False)

lin_60.weight.data = fc_1_m
print(fc_1_m.shape)
lin_8460.weight.data = reduced_fc_2
output = lin_60(random)
output = lin_8460(output)
output = nn.functional.normalize(output, dim=0)
print(output)

lin_120 = nn.Linear(256, 120, bias=False)
lin_84 = nn.Linear(120,84, bias=False)
lin_120.weight.data = model_mnist.fc1.weight
lin_84.weight.data = model_mnist.fc2.weight

output2 = lin_120(random)
output2 = lin_84(output2)


print(output2)

print(nn.CosineSimilarity(dim=0)(output2, output) * 100)


dist = torch.pairwise_distance(output, output2)
print(dist)
result =1/(1 +dist)
print(result *100) 


torch.Size([40, 256])
tensor([ 0.0655, -0.2840,  0.0715, -0.1201,  0.0106, -0.1545, -0.0709, -0.1225,
        -0.2019,  0.0782,  0.1379,  0.0368,  0.1260, -0.1121, -0.0119,  0.0837,
        -0.0479, -0.1236,  0.1093, -0.0095, -0.1081,  0.1483, -0.1773, -0.0478,
         0.0687, -0.0505,  0.0256,  0.0311, -0.0546,  0.0593, -0.1113,  0.1878,
         0.0765, -0.2415, -0.1638, -0.0173,  0.1131,  0.1656,  0.1141, -0.1151,
        -0.0329,  0.0527,  0.0702, -0.1108,  0.1315,  0.0657, -0.1342, -0.0222,
         0.0230,  0.1309, -0.0738, -0.0666, -0.1203, -0.0908,  0.1100,  0.0595,
        -0.0408, -0.0239, -0.0839, -0.0709, -0.0875,  0.0174,  0.1358,  0.1235,
         0.0392, -0.1933,  0.0293, -0.1111,  0.0310, -0.0158,  0.0011,  0.1015,
         0.0247, -0.1355, -0.1582,  0.0930,  0.0432, -0.0255,  0.0324,  0.1620,
        -0.0675,  0.2043, -0.1831, -0.1480], grad_fn=<DivBackward0>)
tensor([ 1.5537,  0.9938, -0.3086, -1.5574,  0.6890, -1.6042, -0.0787, -0.3769,
         0.2473,  0.8930, -1.

In [11]:
new_martixfor_fc2 = torch.mm(fc_2_x.T, fc_2_reduced_m.T)
print(new_martixfor_fc2.shape)

torch.Size([84, 42])


In [12]:
fc_3_new = torch.mm(model_mnist.fc3.weight, new_martixfor_fc2)
print(fc_3_new.shape)
# fc_3_new = model_mnist.fc3.weight.detach().numpy().T
# fc3_Isomap = nonlin.Isomap(n_components=5, n_neighbors=10)
# fc_3_new_reduced = fc3_Isomap.fit_transform(fc_3_new)
# fc_3_new_reduced = fc_3_new_reduced.reshape(-1, 10)
# fc_3_new_reduced = torch.tensor(fc_3_new_reduced.T, dtype=torch.float32)
# print(fc_3_new_reduced.shape)


torch.Size([10, 42])


In [13]:
fc_3_new = torch.mm(fc_3_new, model_mnist.fc3.weight.T)
print(fc_3_new.shape)

torch.Size([42, 10])


In [229]:
print(fc_2_x.shape)

torch.Size([60, 84])


In [13]:
lin1_new = nn.Linear(in_features = 256, out_features=60,bias=True)
lin2_new= nn.Linear(in_features = 60, out_features=42,bias=True)
lin3_new = nn.Linear(in_features = 42, out_features=10,bias=True)

lin1_new.weight = nn.Parameter(fc_1_m)
lin2_new.weight = nn.Parameter(fc_2_reduced_m)
lin3_new.weight = nn.Parameter(fc_3_new)


In [19]:
mnist_lin1 = nn.Linear(in_features = 256, out_features=120,bias=True)
mnist_lin2 = nn.Linear(in_features = 120, out_features=84,bias=True)
mnist_lin3 = nn.Linear(in_features = 84, out_features=10,bias=True)

mnist_lin1.weight = model_mnist.fc1.weight
mnist_lin2.weight = model_mnist.fc2.weight
mnist_lin3.weight = model_mnist.fc3.weight

In [15]:
print(lin2_new.weight.shape)


torch.Size([42, 60])


In [21]:
a1 = lin1_new(random)
a1 = lin2_new(a1)
a1 = lin3_new(a1)
a1 = nn.functional.normalize(a1, dim=0)

a2 = mnist_lin1(random)
a2 = mnist_lin2(a2)
a2 = mnist_lin3(a2)
a2 = nn.functional.normalize(a2, dim=0)


print(a1)
print(a2)
ed = torch.pairwise_distance(a1, a2)
print(ed)
result =1/(1 +ed)
print(result *100) 

tensor([ 0.2364, -0.1304, -0.2433,  0.1608, -0.5722,  0.4288,  0.2356, -0.3944,
         0.3453, -0.0205], grad_fn=<DivBackward0>)
tensor([ 0.1291,  0.2988, -0.5994, -0.2388, -0.2318,  0.3710, -0.2537,  0.4554,
         0.0404, -0.1135], grad_fn=<DivBackward0>)
tensor(1.2903, grad_fn=<NormBackward1>)
tensor(43.6633, grad_fn=<MulBackward0>)


## Pruning of the network

In [40]:
path_mnist_original_network = './isomap_reduced_network/mnist/custom_nn_mnist_isomap_original.pt'
path_mnist_reduced_isomap_retrained = './isomap_reduced_network/mnist/reduced_isomap_retrained.pt'
path_mnist_small_network = './isomap_reduced_network/mnist/small_network_scratch_isomap.pt'
path_mnist_untrained_reduced_network  ='./isomap_reduced_network/mnist/untrained_reduced_network.pt'

In [41]:
path_mnist = './isomap_reduced_network/mnist/custom_nn_mnist_isomap_original.pt'
model_mnist = torch.load(path_mnist)
model_mnist.eval()

NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [45]:
import copy
# pruned_layers = (model_mnist.fc1, model_mnist.fc2, model_mnist.fc3)
# pruned_model_mnist = NN(is_normalize=False)
pruned_model_mnist = copy.deepcopy(model_mnist)


In [44]:
def prune_function(model, amount_to_prune):
    for name,module in model.named_modules():
        if isinstance(module, nn.Linear):
            prune.random_unstructured(module, 'weight', amount=amount_to_prune)
            print(module._forward_pre_hooks)
        

In [46]:
prune_function(pruned_model_mnist, 0.5)

OrderedDict([(15, <torch.nn.utils.prune.RandomUnstructured object at 0x000001A5874A5250>)])
OrderedDict([(16, <torch.nn.utils.prune.RandomUnstructured object at 0x000001A5874A5EE0>)])
OrderedDict([(17, <torch.nn.utils.prune.RandomUnstructured object at 0x000001A5874A56D0>)])


In [47]:
print('model mnist')
averge_accuracy(model_mnist, testloader_mnist)
print('model mnist pruned')
averge_accuracy(pruned_model_mnist, testloader_mnist)

model mnist
average acc: 98.08
model mnist pruned
average acc: 72.32


In [48]:
path_prune_original_network = './isomap_reduced_network/mnist/pruned/pruned_original_network.pt'
torch.save(pruned_model_mnist, path_prune_original_network)

In [49]:
pruned_x_new = pruned_model_mnist.fc1.weight.detach().numpy().T
iso = nonlin.Isomap(n_components=60)
# x_new_1 = x_new_1.reshape(60,2)
pruned_x_new = iso.fit_transform(pruned_x_new)
# x_new_1 = x_new_1.flatten()
# random_matrix = random.reshape(256, 1)
# x_new_1_matrix = x_new_1.reshape(1,60)
print("x_new vector")
print(pruned_x_new.shape)

# new_weight_matrixIguess = np.matmul(random_matrix, x_new_1_matrix)






uuh = torch.mm(pruned_model_mnist.fc1.weight, torch.tensor(pruned_x_new, dtype=torch.float))
print(uuh.shape)
wfc2_pruned = torch.mm(pruned_model_mnist.fc2.weight, uuh)
print(wfc2_pruned.shape)


blah = nonlin.Isomap(n_components=21)
wfc2_r_pruned = wfc2_pruned.reshape(42, -1)
wfc2_r_pruned = blah.fit_transform(wfc2_r_pruned.detach().numpy().T).T
wfc2_r_pruned = wfc2_r_pruned.reshape(-1, 42)
wfc2_r_pruned = torch.tensor(wfc2_r_pruned, dtype=torch.float)
wfc2_r_pruned = wfc2_r_pruned.T
print(wfc2_r_pruned.shape)


wfc3_r_pruned = torch.mm(wfc2_r_pruned, wfc2_pruned.T)
print(wfc3_r_pruned.shape)


wfc3_r_pruned = torch.mm(pruned_model_mnist.fc3.weight, wfc3_r_pruned.T)
print(wfc3_r_pruned.shape)


x_new vector
(256, 60)
torch.Size([120, 60])
torch.Size([84, 60])
torch.Size([42, 60])
torch.Size([42, 84])
torch.Size([10, 42])


In [94]:
pruned_layers = (nn.Linear(256, 60, False), nn.Linear(60,42, False), nn.Linear(42,10, False))
dr_pruned_model_mnist = construct_model(NN(is_normalize=True), pruned_layers)
dr_pruned_model_mnist.fc1.weight.data =  torch.tensor(pruned_x_new.T, dtype=torch.float32)
dr_pruned_model_mnist.fc2.weight.data = wfc2_r_pruned
dr_pruned_model_mnist.fc3.weight.data = wfc3_r_pruned

dr_pruned_model_mnist.eval()

NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=False)
  (fc2): Linear(in_features=60, out_features=42, bias=False)
  (fc3): Linear(in_features=42, out_features=10, bias=False)
  (soft): Softmax(dim=0)
)

In [95]:
averge_accuracy(dr_pruned_model_mnist, testloader_mnist)

average acc: 7.59


In [98]:
x = dr_pruned_model_mnist(sample_im)
x2 = pruned_model_mnist(sample_im)
cos = nn.functional.cosine_similarity(x, x2)
print(cos.sum().item()/4 * 100) 

37.9929780960083


In [99]:
path_dr_prune_original_network = './isomap_reduced_network/mnist/pruned/dr_pruned_original_network.pt'
torch.save(dr_pruned_model_mnist, path_dr_prune_original_network)
print('saved!!')

saved!!


In [100]:
dr_pruned_model_mnist_retrained = copy.deepcopy(dr_pruned_model_mnist)

In [102]:
training(dr_pruned_model_mnist_retrained, trainloader_mnist, 5)

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 1.774 acc: 75.425
[1,  4000] loss: 1.583 acc: 93.362
[1,  6000] loss: 1.521 acc: 95.175
[1,  8000] loss: 1.490 acc: 95.975
[1, 10000] loss: 1.481 acc: 96.650
[1, 12000] loss: 1.470 acc: 97.250
[1, 14000] loss: 1.469 acc: 96.812
epoch: 1
[2,  2000] loss: 2.193 acc: 97.142
[2,  4000] loss: 1.457 acc: 97.375
[2,  6000] loss: 1.455 acc: 97.638
[2,  8000] loss: 1.456 acc: 97.362
[2, 10000] loss: 1.450 acc: 97.912
[2, 12000] loss: 1.449 acc: 97.900
[2, 14000] loss: 1.449 acc: 97.912
epoch: 2
[3,  2000] loss: 2.171 acc: 97.808
[3,  4000] loss: 1.445 acc: 98.138
[3,  6000] loss: 1.445 acc: 98.025
[3,  8000] loss: 1.443 acc: 98.400
[3, 10000] loss: 1.442 acc: 98.350
[3, 12000] loss: 1.444 acc: 97.950
[3, 14000] loss: 1.442 acc: 98.188
epoch: 3
[4,  2000] loss: 2.160 acc: 98.375
[4,  4000] loss: 1.441 acc: 98.162
[4,  6000] loss: 1.439 acc: 98.475
[4,  8000] loss: 1.439 acc: 98.550
[4, 10000] loss: 1.438 acc: 98.688
[4, 12000] loss: 1.440 acc: 98.325
[4, 14000] loss: 1.439 acc: 

# Dimensional reduction on cifar10 dataset

In [169]:
class NN2(nn.Module):
    def __init__(self, is_norm=False):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12,kernel_size=(5,5))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=16, kernel_size=(5,5))
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84, 10)
        self.soft = nn.Softmax(dim=0)
        self.is_norm = is_norm
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)
       



    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        if(self.is_norm == True):
            x = nn.functional.normalize(self.fc3(x))
        else:
            x = self.fc3(x)
        return x


## Original network training

In [124]:
model_cifar = NN2(is_norm=False)
model_cifar.eval()

NN2(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [125]:
#check the accarcy before the training
averge_accuracy(model_cifar, testloader)

average acc: 10.15


In [126]:
training(model_cifar, trainloader, 5)

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0
[1,  2000] loss: 2.189 acc: 17.325
[1,  4000] loss: 1.819 acc: 32.638
[1,  6000] loss: 1.648 acc: 39.450
[1,  8000] loss: 1.563 acc: 42.388
[1, 10000] loss: 1.477 acc: 45.712
[1, 12000] loss: 1.422 acc: 48.150
epoch: 1
[2,  2000] loss: 1.697 acc: 50.640
[2,  4000] loss: 1.323 acc: 52.638
[2,  6000] loss: 1.295 acc: 53.138
[2,  8000] loss: 1.279 acc: 54.200
[2, 10000] loss: 1.254 acc: 55.350
[2, 12000] loss: 1.207 acc: 56.663
epoch: 2
[3,  2000] loss: 1.454 acc: 58.860


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001A580722040>
Traceback (most recent call last):
  File "c:\Users\sulta\anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "c:\Users\sulta\anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 1439, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


KeyboardInterrupt: 

In [117]:
cifar_original_net_path = './isomap_reduced_network/cifar/cifar_original_net.pt'
torch.save(model_cifar, cifar_original_net_path)

## Reduced network training without dimensioal reduction

In [118]:
cifar_layers = (nn.Linear(400, 60), nn.Linear(60, 42), nn.Linear(42,10))
model_cifar_reduced_scratch = construct_model(NN2(is_norm=False), cifar_layers)


In [119]:
# accuracy of reduced network
averge_accuracy(model_cifar_reduced_scratch,testloader)

average acc: 11.03


In [120]:
#train reduced network
training(model_cifar_reduced_scratch, trainloader, 5)

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 2.199 acc: 17.788
[1,  4000] loss: 1.813 acc: 33.388
[1,  6000] loss: 1.665 acc: 38.862
[1,  8000] loss: 1.559 acc: 43.138
[1, 10000] loss: 1.501 acc: 46.263
[1, 12000] loss: 1.466 acc: 46.825
epoch: 1
[2,  2000] loss: 1.726 acc: 49.770
[2,  4000] loss: 1.361 acc: 50.212
[2,  6000] loss: 1.319 acc: 53.337
[2,  8000] loss: 1.298 acc: 53.688
[2, 10000] loss: 1.292 acc: 54.000
[2, 12000] loss: 1.245 acc: 56.337
epoch: 2
[3,  2000] loss: 1.495 acc: 57.640
[3,  4000] loss: 1.203 acc: 58.275
[3,  6000] loss: 1.189 acc: 57.700
[3,  8000] loss: 1.149 acc: 59.212
[3, 10000] loss: 1.188 acc: 58.663
[3, 12000] loss: 1.141 acc: 59.650
epoch: 3
[4,  2000] loss: 1.366 acc: 60.750
[4,  4000] loss: 1.090 acc: 61.788
[4,  6000] loss: 1.080 acc: 62.513
[4,  8000] loss: 1.064 acc: 62.625
[4, 10000] loss: 1.096 acc: 61.050
[4, 12000] loss: 1.080 acc: 62.087
epoch: 4
[5,  2000] loss: 1.269 acc: 64.190
[5,  4000] loss: 1.022 acc: 63.600
[5,  6000] loss: 1.017 acc: 64.213
[5,  8000] loss: 1.

In [121]:
path_reduced_net_scratch = './isomap_reduced_network/cifar/cifar_small_net_scratch.pt'
averge_accuracy(model_cifar_reduced_scratch, testloader)
torch.save(model_cifar_reduced_scratch, path_reduced_net_scratch)

average acc: 62.04


## reduced untrained network

In [122]:
cifar_fc1 = model_cifar.fc1.weight.detach().numpy().T
fa_cifar = nonlin.Isomap(n_components=60, n_neighbors=10)
fc_1_reduced_cifar = fa_cifar.fit_transform(cifar_fc1)
fc_1_reduced_cifar = torch.tensor(fc_1_reduced_cifar.T,  dtype=torch.float32)
print(fc_1_reduced_cifar.shape)



torch.Size([60, 400])


In [123]:
m_projection_60 = torch.mm(model_cifar.fc1.weight,
               fc_1_reduced_cifar.T)
print(m_projection_60.shape)
cifar_wfc2 = torch.mm(model_mnist.fc2.weight, m_projection_60)
print(cifar_wfc2.shape)


torch.Size([120, 60])
torch.Size([84, 60])


In [124]:
cifar_isomap = nonlin.Isomap(n_components=21)
cifar_wfc2_r = cifar_wfc2.reshape(42, -1)
cifar_wfc2_r = cifar_isomap.fit_transform(cifar_wfc2_r.detach().numpy().T).T
cifar_wfc2_r = cifar_wfc2_r.reshape(-1, 42)
cifar_wfc2_r = torch.tensor(cifar_wfc2_r, dtype=torch.float)
cifar_wfc2_r = cifar_wfc2_r.T
print(cifar_wfc2_r.shape)


torch.Size([42, 60])


In [125]:
cifar_wfc3_r = torch.mm(cifar_wfc2_r, cifar_wfc2.T)
print(cifar_wfc3_r.shape)


cifar_wfc3_r = torch.mm(model_cifar.fc3.weight, cifar_wfc3_r.T)
print(cifar_wfc3_r.shape)


torch.Size([42, 84])
torch.Size([10, 42])


In [126]:
cifar_layer2 = (nn.Linear(400, 60), nn.Linear(60, 42), nn.Linear(42,10))
cifar_weights = (fc_1_reduced_cifar, cifar_wfc2_r, cifar_wfc3_r)
cifar_reduced_model_untrained = construct_model(NN2(is_norm=True), cifar_layer2)
cifar_reduced_model_untrained = are_weights(cifar_reduced_model_untrained, cifar_weights)
cifar_reduced_model_untrained.eval()


NN2(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=42, bias=True)
  (fc3): Linear(in_features=42, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [127]:
path_cifar_untrained_dr_network = './isomap_reduced_network/cifar/cifar_untrained_dr_network.pt'
torch.save(cifar_reduced_model_untrained, path_cifar_untrained_dr_network)

## DR reduced network trained

In [128]:
cifar_layer3 = (nn.Linear(400, 60), nn.Linear(60, 42), nn.Linear(42,10))
cifar_reduced_model_trained = construct_model(NN2(is_norm=True), cifar_layer3)
cifar_reduced_model_trained = are_weights(cifar_reduced_model_untrained, cifar_weights)

In [129]:
training(cifar_reduced_model_trained, trainloader, 5)


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 2.210 acc: 19.587
[1,  4000] loss: 2.114 acc: 26.525
[1,  6000] loss: 2.070 acc: 31.750
[1,  8000] loss: 2.059 acc: 32.888
[1, 10000] loss: 2.045 acc: 34.900
[1, 12000] loss: 2.031 acc: 36.062
epoch: 1
[2,  2000] loss: 2.519 acc: 37.790
[2,  4000] loss: 2.013 acc: 38.913
[2,  6000] loss: 2.005 acc: 40.038
[2,  8000] loss: 1.993 acc: 41.788
[2, 10000] loss: 1.993 acc: 41.425
[2, 12000] loss: 1.993 acc: 41.300
epoch: 2
[3,  2000] loss: 2.473 acc: 43.870
[3,  4000] loss: 1.974 acc: 44.500
[3,  6000] loss: 1.974 acc: 44.513
[3,  8000] loss: 1.968 acc: 45.725
[3, 10000] loss: 1.963 acc: 45.925
[3, 12000] loss: 1.964 acc: 45.875
epoch: 3
[4,  2000] loss: 2.449 acc: 47.130
[4,  4000] loss: 1.951 acc: 47.075
[4,  6000] loss: 1.949 acc: 48.013
[4,  8000] loss: 1.951 acc: 47.775
[4, 10000] loss: 1.949 acc: 48.275
[4, 12000] loss: 1.947 acc: 48.362
epoch: 4
[5,  2000] loss: 2.425 acc: 49.760
[5,  4000] loss: 1.941 acc: 49.425
[5,  6000] loss: 1.939 acc: 50.100
[5,  8000] loss: 1.

In [130]:
path_cifar_trained_dr_network = './isomap_reduced_network/cifar/cifar_trained_dr_network.pt'
torch.save(cifar_reduced_model_trained, path_cifar_trained_dr_network)

## Pruning cifar10 model

In [186]:
path_cifar10_original_net = './isomap_reduced_network/cifar/cifar_original_net.pt'
cifar10_model = torch.load(path_cifar10_original_net)
cifar10_model.eval()

NN2(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [187]:
averge_accuracy(cifar10_model, testloader)

average acc: 64.22


In [188]:
cifar10_model_pruned = copy.deepcopy(cifar10_model)
prune_function(cifar10_model_pruned, 0.3)


OrderedDict([(36, <torch.nn.utils.prune.RandomUnstructured object at 0x000001A587461F10>)])
OrderedDict([(37, <torch.nn.utils.prune.RandomUnstructured object at 0x000001A5AC1F8D30>)])
OrderedDict([(38, <torch.nn.utils.prune.RandomUnstructured object at 0x000001A5AC1F8C40>)])


In [189]:
averge_accuracy(cifar10_model, testloader)

averge_accuracy(cifar10_model_pruned, testloader)


average acc: 64.22
average acc: 52.36


In [190]:
cifar10_model_pruned.eval()

NN2(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [191]:
path = './isomap_reduced_network/cifar/pruned/pruned_cifar_original_net.pt'
torch.save(cifar10_model_pruned, path)


In [192]:
cifar_pruned_weight = cifar10_model_pruned.fc1.weight.detach().numpy().T
iso = nonlin.Isomap(n_components=60)
# x_new_1 = x_new_1.reshape(60,2)
cifar_pruned_weight = iso.fit_transform(cifar_pruned_weight)
# x_new_1 = x_new_1.flatten()
# random_matrix = random.reshape(256, 1)
# x_new_1_matrix = x_new_1.reshape(1,60)
print("x_new vector")
print(cifar_pruned_weight.shape)

# new_weight_matrixIguess = np.matmul(random_matrix, x_new_1_matrix)






uuh = torch.mm(cifar10_model_pruned.fc1.weight, torch.tensor(cifar_pruned_weight, dtype=torch.float))
print(uuh.shape)
wfc2_pruned = torch.mm(cifar10_model_pruned.fc2.weight, uuh)
print(wfc2_pruned.shape)


blah = nonlin.Isomap(n_components=21)
wfc2_r_pruned = wfc2_pruned.reshape(42, -1)
wfc2_r_pruned = blah.fit_transform(wfc2_r_pruned.detach().numpy().T).T
wfc2_r_pruned = wfc2_r_pruned.reshape(-1, 42)
wfc2_r_pruned = torch.tensor(wfc2_r_pruned, dtype=torch.float)
wfc2_r_pruned = wfc2_r_pruned.T
print(wfc2_r_pruned.shape)


wfc3_r_pruned = torch.mm(wfc2_r_pruned, wfc2_pruned.T)
print(wfc3_r_pruned.shape)


wfc3_r_pruned = torch.mm(cifar10_model_pruned.fc3.weight, wfc3_r_pruned.T)
print(wfc3_r_pruned.shape)


x_new vector
(400, 60)
torch.Size([120, 60])
torch.Size([84, 60])
torch.Size([42, 60])
torch.Size([42, 84])
torch.Size([10, 42])


In [193]:
cifar_weights = (torch.tensor(cifar_pruned_weight.T, dtype=torch.float32),wfc2_r_pruned,  wfc3_r_pruned)
pruned_and_dr_cifar_network = construct_model(NN2(is_norm=True), (nn.Linear(400, 60), nn.Linear(60, 42), nn.Linear(42,10)))
pruned_and_dr_cifar_network = are_weights(pruned_and_dr_cifar_network, cifar_weights)
pruned_and_dr_cifar_network.eval()

NN2(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=42, bias=True)
  (fc3): Linear(in_features=42, out_features=10, bias=True)
  (soft): Softmax(dim=0)
)

In [194]:
averge_accuracy(pruned_and_dr_cifar_network, testloader)


average acc: 13.43


In [195]:
path_pruned_n_dr_network = './isomap_reduced_network/cifar/pruned/dr_pruned_cifar_network.pt'
torch.save(pruned_and_dr_cifar_network, path_pruned_n_dr_network )